# Distirbuted Training of Mask-RCNN in Amazon SageMaker using S3

This notebook is a step-by-step tutorial on distributed tranining of [Mask R-CNN](https://arxiv.org/abs/1703.06870) implemented in [TensorFlow](https://www.tensorflow.org/) framework. Mask R-CNN is also referred to as heavy weight object detection model and it is part of [MLPerf](https://www.mlperf.org/training-results-0-6/).

Concretely, we will describe the steps for training [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) and [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) in [Amazon SageMaker](https://aws.amazon.com/sagemaker/) using [Amazon S3](https://aws.amazon.com/s3/) as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Build SageMaker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
3. Configure data input channels
4. Configure hyper-prarameters
5. Define training metrics
6. Define training job and start training

Before we get started, let us initialize two python variables ```aws_region``` and ```s3_bucket``` that we will use throughout the notebook:

In [1]:
aws_region =  "ap-southeast-1"
s3_bucket  =  "smart-invoice"

## Stage COCO 2017 dataset in Amazon S3

We use [COCO 2017 dataset](http://cocodataset.org/#home) for training. We download COCO 2017 training and validation dataset to this notebook instance, extract the files from the dataset archives, and upload the extracted files to your Amazon [S3 bucket](https://docs.aws.amazon.com/en_pv/AmazonS3/latest/gsg/CreatingABucket.html) with the prefix ```mask-rcnn/sagemaker/input/train```. The ```prepare-s3-bucket.sh``` script executes this step.


In [46]:
!cat ./prepare-s3-bucket.sh

#!/bin/bash

set -e

if [ "$#" -ne 1 ]; then
    echo "usage: $0 <s3-bucket-name>"
    exit 1
fi

S3_BUCKET=$1
S3_PREFIX="mask-rcnn/sagemaker/input"

# Stage directory must be on EBS volume with 100 GB available space
STAGE_DIR=$HOME/SageMaker/coco-2017-$(date +"%Y-%m-%d-%H-%M-%S")

echo "Create stage directory: $STAGE_DIR"
mkdir -p $STAGE_DIR

wget -O $STAGE_DIR/train2017.zip http://images.cocodataset.org/zips/train2017.zip
echo "Extracting $STAGE_DIR/train2017.zip"
unzip -o $STAGE_DIR/train2017.zip  -d $STAGE_DIR | awk 'BEGIN {ORS="="} {if(NR%1000==0)print "="}'
echo "Done."
rm $STAGE_DIR/train2017.zip

wget -O $STAGE_DIR/val2017.zip http://images.cocodataset.org/zips/val2017.zip
echo "Extracting $STAGE_DIR/val2017.zip"
unzip -o $STAGE_DIR/val2017.zip -d $STAGE_DIR | awk 'BEGIN {ORS="="} {if(NR%1000==0)print "="}'
echo "Done."
rm $STAGE_DIR/val2017.zip

wget -O $STAGE_DIR/annotations_trainval2017.zip http://images.cocodataset.org/annotations/annotations_trainval2017.zip
unzip -o $STA

 Using your *Amazon S3 bucket* as argument, run the cell below. If you have already uploaded COCO 2017 dataset to your Amazon S3 bucket *in this AWS region*, you may skip this step. The expected time to execute this step is 20 minutes.

In [ ]:
%%time
!./prepare-s3-bucket.sh {s3_bucket}

Create stage directory: /home/ec2-user/SageMaker/coco-2017-2020-04-02-07-16-41
--2020-04-02 07:16:41--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.142.124
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.142.124|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘/home/ec2-user/SageMaker/coco-2017-2020-04-02-07-16-41/train2017.zip’

Maker/coco-2017-202  20%[===>                ]   3.74G  6.90MB/s    eta 36m 30s

## Build and push SageMaker training images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we have a choice of two different implementations:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) implementation supports a maximum per-GPU batch size of 1, and does not support mixed precision. It can be used with mainstream TensorFlow releases.

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) is an optimized implementation that supports a maximum batch size of 4 and supports mixed precision. This implementation uses TensorFlow base version 1.13 augmented with custom TensorFlow ops. 

It is recommended that you build and push both SageMaker training images and use either image for training later.


### TensorPack Faster-RCNN/Mask-RCNN

Use ```./container/build_tools/build_and_push.sh``` script to build and push the TensorPack Faster-RCNN/Mask-RCNN  training image to Amazon ECR. 

In [4]:
!cat ./container/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi
  

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container/build_tools/build_and_push.sh {aws_region}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  17.92kB
Step 1/24 : FROM  763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:1.13-horovod-gpu-py36-cu100-ubuntu16.04-v2.0
1.13-horovod-gpu-py36-cu100-ubuntu16.04-v2.0: Pulling from tensorflow-training

e2e5f967: Pulling fs layer 
6638ac9f: Pulling fs layer 
7d6d954b: Pulling fs layer 
e5e2a643: Pulling fs layer 
43379044: Pulling fs layer 
c12a6c3b: Pulling fs layer 
93d9ac61: Pulling fs layer 
a7b48be6: Pulling fs layer 
eecb5348: Pulling fs layer 
6e6c5b44: Pulling fs layer 
b7e5eee6: Pulling fs layer 
f1cf8b04: Pulling fs layer 
2ded661d: Pulling fs layer 
51eff8ef: Pulling fs layer 
b6a3e9a7: Pulling fs layer 
14a60d05:

 ---> Running in ad0abf2e96a1
Reading package lists...
Building dependency tree...
Reading state information...
openssh-client is already the newest version (1:7.2p2-4ubuntu2.8).
openssh-server is already the newest version (1:7.2p2-4ubuntu2.8).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Removing intermediate container ad0abf2e96a1
 ---> cb410f418202
Step 8/24 : RUN mkdir -p /var/run/sshd &&   sed 's@session\s*required\s*pam_loginuid.so@session optional pam_loginuid.so@g' -i /etc/pam.d/sshd
 ---> Running in d475ed3fba95
Removing intermediate container d475ed3fba95
 ---> 60ec9899bb09
Step 9/24 : RUN rm -rf /root/.ssh/ &&   mkdir -p /root/.ssh/ &&   ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa &&   cp /root/.ssh/id_rsa.pub /root/.ssh/authorized_keys &&   printf "Host *\n  StrictHostKeyChecking no\n" >> /root/.ssh/config
 ---> Running in 0e1f9a9dab8c
Removing intermediate container 0e1f9a9dab8c
 ---> 7d0630869706
Step 10/24 : RUN pip install awscli
 ---> Running in

Removing intermediate container b324130f6f9a
 ---> a4ce37a055c8
Step 17/24 : RUN pip install markdown==3.1
 ---> Running in fb96910d6ae9
  Attempting uninstall: markdown
    Found existing installation: Markdown 3.2.1
    Uninstalling Markdown-3.2.1:
      Successfully uninstalled Markdown-3.2.1
Removing intermediate container fb96910d6ae9
 ---> d62d8a6ccb91
Step 18/24 : RUN pip install numpy==1.17.5
 ---> Running in f0cc1ca3fa3d
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
Removing intermediate container f0cc1ca3fa3d
 ---> 0056a2f9b800
Step 19/24 : RUN git clone https://github.com/tensorpack/tensorpack.git /tensorpack
 ---> Running in a80d4cf73a99
Cloning into '/tensorpack'...
Removing intermediate container a80d4cf73a99
 ---> 41c0d207f8b9
Step 20/24 : RUN cd /tensorpack && git fetch origin 26664c3f1d58ae029ea6c3ba0af6ae11900b1e55
 ---> Running in 1dcc7763e55d
From https://github.

Set ```tensorpack_image``` below to Amazon ECR URI of the image you pushed above.

In [5]:
tensorpack_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-sagemaker:tf1.13-tp26664c3"

### AWS Samples Mask R-CNN
Use ```./container-optimized/build_tools/build_and_push.sh``` script to build and push the AWS Samples Mask R-CNN training image to Amazon ECR.

In [22]:
!cat ./container-optimized/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi
  

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${

Using your *AWS region* as argument, run the cell below.

In [34]:
%%time
! ./container-optimized/build_tools/build_and_push.sh {aws_region}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  18.94kB
Step 1/25 : FROM  tensorflow-base:1.13.1-gpu-py36
 ---> 678566651313
Step 2/25 : ENV HOROVOD_VERSION=0.18.1
 ---> Running in 9ee0d7738a21
Removing intermediate container 9ee0d7738a21
 ---> bf5565ec2297
Step 3/25 : RUN pip install --upgrade pip
 ---> Running in 2d5642321d7d
Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (20.0.2)
Removing intermediate container 2d5642321d7d
 ---> 4622fa702605
Step 4/25 : RUN pip uninstall -y tensorflow tensorboard tensorflow-estimator keras h5py horovod numpy
 ---> Running in da760f26c281
Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
 

  1950K .......... .......... .......... .......... ..........  1%  227K 1m45s
  2000K .......... .......... .......... .......... ..........  1% 99.1M 1m42s
  2050K .......... .......... .......... .......... ..........  1%  109M 1m40s
  2100K .......... .......... .......... .......... ..........  1% 25.4M 97s
  2150K .......... .......... .......... .......... ..........  1% 80.1M 95s
  2200K .......... .......... .......... .......... ..........  1% 30.6M 93s
  2250K .......... .......... .......... .......... ..........  1% 21.8M 91s
  2300K .......... .......... .......... .......... ..........  1% 16.2M 89s
  2350K .......... .......... .......... .......... ..........  1% 36.3M 88s
  2400K .......... .......... .......... .......... ..........  1% 51.8M 86s
  2450K .......... .......... .......... .......... ..........  1%  168M 84s
  2500K .......... .......... .......... .......... ..........  1%  188M 82s
  2550K .......... .......... .......... .......... ..........  1%  17

  6250K .......... .......... .......... .......... ..........  4%  230K 46s
  6300K .......... .......... .......... .......... ..........  4%  164M 46s
  6350K .......... .......... .......... .......... ..........  4%  176M 45s
  6400K .......... .......... .......... .......... ..........  4%  142M 45s
  6450K .......... .......... .......... .......... ..........  4%  170M 44s
  6500K .......... .......... .......... .......... ..........  5% 20.0M 44s
  6550K .......... .......... .......... .......... ..........  5% 38.5M 44s
  6600K .......... .......... .......... .......... ..........  5% 83.9M 43s
  6650K .......... .......... .......... .......... ..........  5% 61.6M 43s
  6700K .......... .......... .......... .......... ..........  5% 36.2M 43s
  6750K .......... .......... .......... .......... ..........  5% 93.2M 43s
  6800K .......... .......... .......... .......... ..........  5%  123M 42s
  6850K .......... .......... .......... .......... ..........  5% 82.9M 42s

 10550K .......... .......... .......... .......... ..........  8% 87.9M 32s
 10600K .......... .......... .......... .......... ..........  8%  137M 32s
 10650K .......... .......... .......... .......... ..........  8% 85.7M 31s
 10700K .......... .......... .......... .......... ..........  8% 73.4M 31s
 10750K .......... .......... .......... .......... ..........  8%  127M 31s
 10800K .......... .......... .......... .......... ..........  8%  167M 31s
 10850K .......... .......... .......... .......... ..........  8%  230K 33s
 10900K .......... .......... .......... .......... ..........  8%  149M 33s
 10950K .......... .......... .......... .......... ..........  8%  179M 33s
 11000K .......... .......... .......... .......... ..........  8% 38.0M 33s
 11050K .......... .......... .......... .......... ..........  8%  162M 33s
 11100K .......... .......... .......... .......... ..........  8%  177M 32s
 11150K .......... .......... .......... .......... ..........  8%  186M 32s

 15450K .......... .......... .......... .......... .......... 11%  234K 28s
 15500K .......... .......... .......... .......... .......... 11% 80.7M 28s
 15550K .......... .......... .......... .......... .......... 11% 14.5M 27s
 15600K .......... .......... .......... .......... .......... 12% 71.1M 27s
 15650K .......... .......... .......... .......... .......... 12%  159M 27s
 15700K .......... .......... .......... .......... .......... 12%  168M 27s
 15750K .......... .......... .......... .......... .......... 12%  176M 27s
 15800K .......... .......... .......... .......... .......... 12%  147M 27s
 15850K .......... .......... .......... .......... .......... 12%  176M 27s
 15900K .......... .......... .......... .......... .......... 12%  167M 27s
 15950K .......... .......... .......... .......... .......... 12%  164M 27s
 16000K .......... .......... .......... .......... .......... 12% 14.1M 27s
 16050K .......... .......... .......... .......... .......... 12%  187M 27s

...

125900K .......... .......... .......... .......... .......... 96%  261K 1s
125950K .......... .......... .......... .......... .......... 96% 16.7M 1s
126000K .......... .......... .......... .......... .......... 96% 25.3M 1s
126050K .......... .......... .......... .......... .......... 96% 64.5M 1s
126100K .......... .......... .......... .......... .......... 96% 55.9M 1s
126150K .......... .......... .......... .......... .......... 96% 48.4M 1s
126200K .......... .......... .......... .......... .......... 96% 19.7M 1s
126250K .......... .......... .......... .......... .......... 97%  156M 1s
126300K .......... .......... .......... .......... .......... 97% 12.5M 1s
126350K .......... .......... .......... .......... .......... 97% 26.5M 1s
126400K .......... .......... .......... .......... .......... 97% 27.7M 1s
126450K .......... .......... .......... .......... .......... 97% 72.6M 1s
126500K .......... .......... .......... .......... .......... 97% 14.1M 1s
126550K ....

129500K .......... .......... .......... .......... .......... 99% 67.7M 0s
129550K .......... .......... .......... .......... .......... 99% 13.4M 0s
129600K .......... .......... .......... .......... .......... 99% 57.5M 0s
129650K .......... .......... .......... .......... .......... 99% 55.1M 0s
129700K .......... .......... .......... .......... .......... 99% 39.2M 0s
129750K .......... .......... .......... .......... .......... 99% 20.4M 0s
129800K .......... .......... .......... .......... .......... 99% 34.2M 0s
129850K .......... .......... .......... .......... .......... 99%  176M 0s
129900K .......... .......... .......... .......... .......... 99%  181M 0s
129950K .......... .......... .......... .......... .......... 99%  183M 0s
130000K .......... .......... .......... .......... .......... 99% 77.2M 0s
130050K .......... .......... .......... .......... .......... 99% 55.2M 0s
130100K .......... .......... .......... .......... .......... 99%  142M 0s
130150K ....

 ---> Running in 1f2d94d2e7e4
  Created wheel for PyYAML: filename=PyYAML-3.13-cp36-cp36m-linux_x86_64.whl size=43086 sha256=641222c49a7514663f997cd5870998c97ded445fcd8f1cec6ba61525c3ce3201
  Stored in directory: /root/.cache/pip/wheels/ab/80/60/97eaebe423482bcf0963b19495b4cfe6c4caef59761a0c427d
Successfully built PyYAML
ERROR: boto3 1.12.34 has requirement botocore<1.16.0,>=1.15.34, but you'll have botocore 1.12.120 which is incompatible.
ERROR: boto3 1.12.34 has requirement s3transfer<0.4.0,>=0.3.0, but you'll have s3transfer 0.2.1 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.15.34
    Uninstalling botocore-1.15.34:
      Successfully uninstalled botocore-1.15.34
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.3
    Uninstalling s3transfer-0.3.3:
      Successfully uninstalled s3transfer-0.3.3
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
    Uninstalling PyYAML-5.

  Attempting uninstall: markdown
    Found existing installation: Markdown 3.2.1
    Uninstalling Markdown-3.2.1:
      Successfully uninstalled Markdown-3.2.1
Removing intermediate container 1476096f321c
 ---> 78c4ed0a5928
Step 18/25 : RUN pip install numpy==1.17.5
 ---> Running in 48440c4686be
ERROR: sagemaker-containers 2.8.6 has requirement scipy>=1.2.2, but you'll have scipy 1.2.1 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2
Removing intermediate container 48440c4686be
 ---> f6cfee78393a
Step 19/25 : RUN git clone https://github.com/aws-samples/mask-rcnn-tensorflow
 ---> Running in 95343baa5d66
Cloning into 'mask-rcnn-tensorflow'...
Removing intermediate container 95343baa5d66
 ---> e0aab41e3f64
Step 20/25 : RUN cd /mask-rcnn-tensorflow && git fetch origin 153442bc70b06e59f2bbeadc4d359b240f64cbc2
 ---> Running in 0b4c9de7f319
From https://github.com/aws-

4bb239a: Pushing  361.6MB/1.098GB40A4bb239a: Pushed   1.115GB/1.098GBtf1.13-153442b: digest: sha256:b28dc5170a5b19cd9e59cf7cb8278eae972554e1bc97573e1238babee642c317 size: 9340
Amazon ECR URI: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b
CPU times: user 8.5 s, sys: 1.6 s, total: 10.1 s
Wall time: 5min 16s


 Set ```aws_samples_image``` below to Amazon ECR URI of the image you pushed above.

In [4]:
aws_samples_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b"

## SageMaker Initialization 
We have staged the data and we have built and pushed the training docker image to Amazon ECR. Now we are ready to start using Amazon SageMaker.

In [2]:
%%time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

SageMaker Execution Role:arn:aws:iam::393782509758:role/service-role/AmazonSageMaker-ExecutionRole-20191115T140457
AWS account:393782509758
AWS region:ap-southeast-1
CPU times: user 831 ms, sys: 103 ms, total: 934 ms
Wall time: 1.9 s


Next, we set ```training_image``` to the Amazon ECR image URI you saved in a previous step. 

In [5]:
training_image =  aws_samples_image# set to tensorpack_image or aws_samples_image 
print(f'Training image: {training_image}')

Training image: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b


## Define SageMaker Data Channels
In this step, we define SageMaker *train* data channel. 

In [6]:
prefix = "mask-rcnn/sagemaker" #prefix in your S3 bucket

s3train = f's3://{s3_bucket}/{prefix}/input/train'


train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/tfrecord', s3_data_type='S3Prefix')


data_channels = {'train': train}

Next, we define the model output location in S3 bucket.

In [7]:
s3_output_location = f's3://{s3_bucket}/{prefix}/output'

# Configure Hyper-parameters
Next, we define the hyper-parameters. 

Note, some hyper-parameters are different between the two implementations. The batch size per GPU in TensorPack Faster-RCNN/Mask-RCNN is fixed at 1, but is configurable in AWS Samples Mask-RCNN. The learning rate schedule is specified in units of steps in TensorPack Faster-RCNN/Mask-RCNN, but in epochs in AWS Samples Mask-RCNN.

The detault learning rate schedule values shown below correspond to training for a total of 24 epochs, at 120,000 images per epoch.

<table align='left'>
    <caption>TensorPack Faster-RCNN/Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_schedule</td>
        <td style="text-align:left">Learning rate schedule in training steps</td>
        <td style="text-align:center">'[240000, 320000, 360000]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'coco_train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'coco_val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

    
<table align='left'>
    <caption>AWS Samples Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_epoch_schedule</td>
        <td style="text-align:left">Learning rate schedule in epochs</td>
        <td style="text-align:center">'[(16, 0.1), (20, 0.01), (24, None)]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_size_per_gpu</td>
        <td style="text-align:left">Batch size per gpu ( Minimum 1, Maximum 4)</td>
        <td style="text-align:center">4</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

In [9]:
hyperparameters = {
                    "mode_fpn": "True",
                    "mode_mask": "True",
                    "eval_period": 1,
                    "batch_norm": "FreezeBN",
                    "batch_size_per_gpu": 1,
                  }

## Define Training Metrics
Next, we define the regular expressions that SageMaker uses to extract algorithm metrics from training logs and send them to [AWS CloudWatch metrics](https://docs.aws.amazon.com/en_pv/AmazonCloudWatch/latest/monitoring/working_with_metrics.html). These algorithm metrics are visualized in SageMaker console.

In [10]:
metric_definitions=[
             {
                "Name": "fastrcnn_losses/box_loss",
                "Regex": ".*fastrcnn_losses/box_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_loss",
                "Regex": ".*fastrcnn_losses/label_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/false_negative",
                "Regex": ".*fastrcnn_losses/label_metrics/false_negative:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/fg_accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/fg_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/num_fg_label",
                "Regex": ".*fastrcnn_losses/num_fg_label:\\s*(\\S+).*"
            },
             {
                "Name": "maskrcnn_loss/accuracy",
                "Regex": ".*maskrcnn_loss/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/fg_pixel_ratio",
                "Regex": ".*maskrcnn_loss/fg_pixel_ratio:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/maskrcnn_loss",
                "Regex": ".*maskrcnn_loss/maskrcnn_loss:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/pos_accuracy",
                "Regex": ".*maskrcnn_loss/pos_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.75",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/large",
                "Regex": ".*mAP\\(bbox\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/medium",
                "Regex": ".*mAP\\(bbox\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/small",
                "Regex": ".*mAP\\(bbox\\)/small:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.75",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/large",
                "Regex": ".*mAP\\(segm\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/medium",
                "Regex": ".*mAP\\(segm\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/small",
                "Regex": ".*mAP\\(segm\\)/small:\\s*(\\S+).*"
            }  
            
    ]

## Define SageMaker Training Job

Next, we use SageMaker [Estimator](https://sagemaker.readthedocs.io/en/stable/estimators.html) API to define a SageMaker Training Job. 

We recommned using 32 GPUs, so we set ```train_instance_count=4``` and ```train_instance_type='ml.p3.16xlarge'```, because there are 8 Tesla V100 GPUs per ```ml.p3.16xlarge``` instance. We recommend using 100 GB [Amazon EBS](https://aws.amazon.com/ebs/) storage volume with each training instance, so we set ```train_volume_size = 100```. We want to replicate training data to each training instance, so we set ```input_mode= 'File'```.

We run the training job in your private VPC, so we need to set the ```subnets``` and ```security_group_ids``` prior to running the cell below. You may specify multiple subnet ids in the ```subnets``` list. The subnets included in the ```sunbets``` list must be part of the output of  ```./stack-sm.sh``` CloudFormation stack script used to create this notebook instance. Specify only one security group id in ```security_group_ids``` list. The security group id must be part of the output of  ```./stack-sm.sh``` script.

For ```train_instance_type``` below, you have the option to use ```ml.p3.16xlarge``` with 16 GB per-GPU memory and 25 Gbs network interconnectivity, or ```ml.p3dn.24xlarge``` with 32 GB per-GPU memory and 100 Gbs network interconnectivity. The ```ml.p3dn.24xlarge``` instance type offers significantly better performance than ```ml.p3.16xlarge``` for Mask R-CNN distributed TensorFlow training.

In [17]:
# security_group_ids =  # ['sg-xxxxxxxx']
# subnets =      # [ 'subnet-xxxxxxx']
sagemaker_session = sagemaker.session.Session(boto_session=session)
mask_rcnn_estimator = Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p2.xlarge',
                                         #train_instance_type='local_gpu',
                                         train_volume_size = 100,
                                         train_max_run = 400000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker_session, 
                                         hyperparameters = hyperparameters,
                                         metric_definitions = metric_definitions,
                                         base_job_name="mask-rcnn-s3")



Finally, we launch the SageMaker training job. 

The estimated time for downloading data to all the training instances is 20 minutes. The time to complete the training depends on type and number of training instances, and the training image used for training.

In [ ]:
mask_rcnn_estimator.fit(inputs=data_channels, logs="All")

2020-04-02 09:03:20 Starting - Starting the training job...
2020-04-02 09:03:21 Starting - Launching requested ML instances......
2020-04-02 09:04:23 Starting - Preparing the instances for training......
2020-04-02 09:05:31 Downloading - Downloading input data............../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym 


2020-04-02 09:23:33 Training - Training image download completed. Training in progress.[1,0]<stdout>:Done (t=21.33s)
[1,0]<stdout>:creating index...
[1,0]<stdout>:index created!
[1,0]<stdout>:#033[0402 09:24:06 @dataset.py:50]#033 Instances loaded from /opt/ml/input/data/train/annotations/instances_train2017.json.
[1,0]<stdout>:#033[0402 09:26:10 @timer.py:50]#033 Load Groundtruth Boxes for train2017 finished, time:124.1166sec.
[1,0]<stdout>:Done loading roidbs
[1,0]<stdout>:#033[0402 09:26:12 @data.py:509]#033 Filtered 1021 images which contain no non-crowd groudtruth boxes. Total #images for training: 117266
[1,0]<stdout>:Batching roidbs
[1,0]<stdout>:Done batching roidbs
[1,0]<stdout>:#033[0402 09:26:17 @trainers.py:393]#033 [HorovodTrainer] local rank=0
[1,0]<stdout>:#033[0402 09:26:17 @input_source.py:222]#033 Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1,0]<stdout>:#033[0402 09:26:17 @registry.py:127]#033 conv0 input: [None, 3, None, None]
[1,0]<stdout

[1,0]<stdout>:#033[0402 09:26:30 @model_utils.py:66]#033 #033Trainable Variables:
[1,0]<stdout>:#033name                                   shape                    dim
[1,0]<stdout>:-------------------------------------  ------------------  --------
[1,0]<stdout>:group1/block0/conv1/W:0                [1, 1, 256, 128]       32768
[1,0]<stdout>:group1/block0/conv1/bn/gamma:0         [128]                    128
[1,0]<stdout>:group1/block0/conv1/bn/beta:0          [128]                    128
[1,0]<stdout>:group1/block0/conv2/W:0                [3, 3, 128, 128]      147456
[1,0]<stdout>:group1/block0/conv2/bn/gamma:0         [128]                    128
[1,0]<stdout>:group1/block0/conv2/bn/beta:0          [128]                    128
[1,0]<stdout>:group1/block0/conv3/W:0                [1, 1, 128, 512]       65536
[1,0]<stdout>:group1/block0/conv3/bn/gamma:0         [512]                    512
[1,0]<stdout>:group1/block0/conv3/bn/beta:0          [512]                    512
[1,0]<stdout

[1,0]<stdout>:#033[0402 09:26:35 @tower.py:134]#033 Building graph for predict tower 'tower-pred-0' on device /gpu:0 ...
[1,0]<stdout>:c2345 (<tf.Tensor 'tower-pred-0/group0/block2/output:0' shape=(?, 256, ?, ?) dtype=float16>, <tf.Tensor 'tower-pred-0/group1/block3/output:0' shape=(?, 512, ?, ?) dtype=float16>, <tf.Tensor 'tower-pred-0/group2/block5/output:0' shape=(?, 1024, ?, ?) dtype=float16>, <tf.Tensor 'tower-pred-0/group3/block2/output:0' shape=(?, 2048, ?, ?) dtype=float16>)
[1,0]<stdout>:#033[0402 09:26:45 @collection.py:153]#033 Size of these collections were changed in tower-pred-0: (tf.GraphKeys.MODEL_VARIABLES: 183->238)
[1,0]<stdout>:loading annotations into memory...
[1,0]<stdout>:Done (t=0.63s)
[1,0]<stdout>:creating index...
[1,0]<stdout>:index created!
[1,0]<stdout>:#033[0402 09:26:46 @dataset.py:50]#033 Instances loaded from /opt/ml/input/data/train/annotations/instances_val2017.json.
[1,0]<stdout>:#033[0402 09:26:46 @timer.py:50]#033 Load Groundtruth Boxes for val20